# Setup

## Imports

In [62]:
import QuantumToolbox as qt
using Logging
import CairoMakie as cm
using MiniLoggers
using Revise
using Dates

using Symbolics

import NonlinearSolve as NLS

In [151]:
import SuperconductingCavities as SC

## Some Extra Stuff

In [3]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(DebugLogger)

Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(21) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

# Testing SNAIL!

In [45]:
function c(n)
    @variables ϕ α N Φₑ

    U = -α*cos(ϕ) - N * cos((ϕ - Φₑ)/N)

    D = Differential(ϕ)^n

    return expand_derivatives(D(U))
end

c (generic function with 1 method)

In [124]:
function get_c_coeff_bare(N_val, α_val, Φₑ_val)
    @variables ϕ N α Φₑ

    c_syms = []
    for n in 1:6
        c_sym = c(n)
        push!(c_syms, c_sym)
    end

    f_to_min(ϕ_val, p) = [Symbolics.value(substitute(c_syms[1], Dict(ϕ=>ϕ_val[1], N=>N_val, α => α_val, Φₑ => Φₑ_val)))]

    prob = NLS.NonlinearProblem(f_to_min, [0.0], [])
    ϕ_min = NLS.solve(prob)[1]
    @debug "ϕ_min: $ϕ_min"


    cs = []
    for n in 2:6
        push!(cs, Symbolics.value(substitute(c_syms[n], Dict(ϕ=>ϕ_min, N=>N_val, α=>α_val, Φₑ=>Φₑ_val))))
    end

    return cs
end

get_c_coeff_bare (generic function with 1 method)

In [126]:
cs = get_c_coeff_bare(3.0, 0.147, 0.35)

[2024-10-01 17:16:07] Debug: ϕ_min: 0.2436066018139098


5-element Vector{Any}:
  0.4757834649215299
 -0.03151736215001946
 -0.1796734792230421
  0.0350192912777994
  0.1467723697009879

In [121]:
f_to_min([0.0],[0.0])

1-element Vector{Float64}:
 -0.11640218623064896

In [123]:
println(f_to_min([0],[]))

prob = NLS.NonlinearProblem(f_to_min, [0.0], [])

sol = NLS.solve(prob)#, NLS.Klement())

[-0.11640218623064896]


retcode: Success
u: 1-element Vector{Float64}:
 0.2436066018139098

In [97]:
fu(u, p) = u .* u .- p
u0 = [1.0, 1.0]
p = [2.0]
prob = NLS.NonlinearProblem(fu, u0, p)
sol = NLS.solve(prob)

retcode: Success
u: 2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

In [101]:
similar([1])

1-element Vector{Int64}:
 140006393846224

In [150]:
a = qt.create(3)
a+= qt.create(3)

Quantum Object:   type=Operator   dims=[3]   size=(3, 3)   ishermitian=false
3×3 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
     ⋅              ⋅          ⋅    
 2.0+0.0im          ⋅          ⋅    
     ⋅      2.82843+0.0im      ⋅    

In [149]:
num = 2
for i in 3:6
    num *= i
    println(num)
end

6
24
120
720


In [198]:
module A
    a = 2
    module B
        using ..A: a
        b = 1*a
    end
    B.b = 3
end


Main.A

In [199]:
A.B.b

3